In [2]:
import pandas as pd
import numpy as np
from custom_xgoost.xgboost_greedy_split import CustomXGBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

df = pd.read_csv("data/kc_house_data.csv")
df = df.apply(pd.to_numeric, errors='coerce').dropna(axis=1, how='all')

y = df["price"].values
X = df.drop(columns=["price"]).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb = CustomXGBoostRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)
xgb.fit(X_train, y_train)

y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)

train_mse = np.mean((y_train - y_train_pred) ** 2)
test_mse = np.mean((y_test - y_test_pred) ** 2)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Train MSE:", train_mse)
print("Test MSE:", test_mse)
print()
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
print()
print("Train R²:", train_r2)
print("Test R²:", test_r2)


Train MSE: 15089479433.889954
Test MSE: 24936367875.353508

Train RMSE: 122839.24223915562
Test RMSE: 157912.53235685098

Train R²: 0.8813157823882112
Test R²: 0.8500324167994462
